In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rlway.pyosrd import OSRD

In [3]:
OSRD.use_cases

['cvg_dvg', 'divergence', 'station_capacity2']

In [13]:
sim = OSRD(use_case='station_capacity2')

simulation:  Train schedule creation: couldn't find a path
[15:30:28,958] [DEBUG]          [BlockBuilder] unterminated block at end of route rt.D0->D2
[15:30:28,970] [DEBUG]          [BlockBuilder] unterminated block at end of route rt.D0->D1
[15:30:28,973] [DEBUG]          [BlockBuilder] no signal at non buffer stop
[15:30:28,974] [DEBUG]          [BlockBuilder] no signal at non buffer stop
[15:30:28,974] [DEBUG]          [BlockBuilder] no signal at non buffer stop
[15:30:28,975] [DEBUG]          [BlockBuilder] no signal at non buffer stop
[15:30:28,976] [DEBUG]          [BlockBuilder] unterminated block at end of route rt.D5->D4
[15:30:28,976] [DEBUG]          [BlockBuilder] unterminated block at end of route rt.D5->D3


Exception in thread "main" fr.sncf.osrd.railjson.parser.exceptions.InvalidSchedule: Cannot invoke "fr.sncf.osrd.infra.api.signaling.SignalingRoute.getInfraRoute()" because "signalingRoute" is null
	at fr.sncf.osrd.infra_state.implementation.TrainPathBuilder.from(TrainPathBuilder.java:32)
	at fr.sncf.osrd.cli.StandaloneSimulationCommand.run(StandaloneSimulationCommand.java:121)
	at fr.sncf.osrd.App.main(App.java:44)


In [6]:
sim.infra['buffer_stops']

[{'track': 'T0', 'position': 0.0, 'id': 'buffer_stop.0'},
 {'track': 'T5', 'position': 1000.0, 'id': 'buffer_stop.5'}]

In [7]:
sim.infra['track_section_links']

[{'id': 'link.6',
  'src': {'endpoint': 'END', 'track': 'T0'},
  'dst': {'endpoint': 'BEGIN', 'track': 'T1'}},
 {'id': 'link.7',
  'src': {'endpoint': 'END', 'track': 'T0'},
  'dst': {'endpoint': 'BEGIN', 'track': 'T2'}},
 {'id': 'link.8',
  'src': {'endpoint': 'END', 'track': 'T1'},
  'dst': {'endpoint': 'BEGIN', 'track': 'T3'}},
 {'id': 'link.9',
  'src': {'endpoint': 'END', 'track': 'T2'},
  'dst': {'endpoint': 'BEGIN', 'track': 'T4'}},
 {'id': 'link.10',
  'src': {'endpoint': 'BEGIN', 'track': 'T5'},
  'dst': {'endpoint': 'END', 'track': 'T3'}},
 {'id': 'link.11',
  'src': {'endpoint': 'BEGIN', 'track': 'T5'},
  'dst': {'endpoint': 'END', 'track': 'T4'}}]

In [14]:
sim.routes

['rt.D0->D2',
 'rt.D0->D1',
 'rt.buffer_stop.0->D0',
 'rt.D1->buffer_stop.0',
 'rt.D2->buffer_stop.0',
 'rt.D3->buffer_stop.5',
 'rt.D4->buffer_stop.5',
 'rt.D5->D4',
 'rt.D5->D3',
 'rt.buffer_stop.5->D5']

In [ ]:
sim.routes

In [ ]:
sim.points_encountered_by_train(0)